In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("data/sodlokka.txt", delim_whitespace=True, skiprows=[1], na_values=["-9999"])
data = data.reset_index()
data = data[["DATE", "PRCP", "TMAX", "TMIN"]]
data.head()

,DATE,PRCP,TMAX,TMIN
0,19590101,0.03,NaN,9.0
1,19590102,0.00,NaN,6.0
2,19590103,0.02,NaN,-9.0
3,19590104,0.08,NaN,10.0
4,19590105,0.09,NaN,13.0


In [3]:
data = data.dropna()
data["TMAX"] = data["TMAX"].round(0).astype(int)
data["TMIN"] = data["TMIN"].round(0).astype(int)
data["TAVG"] = (data["TMAX"] + data["TMIN"])/2
data["TEMP_C"] = (data['TAVG'] -32) / 1.8
data.head()


,DATE,PRCP,TMAX,TMIN,TAVG,TEMP_C
120,19590501,0.01,53,36,44.5,6.944444
121,19590502,0.00,54,36,45.0,7.222222
122,19590503,0.00,62,41,51.5,10.833333
123,19590504,0.00,63,46,54.5,12.500000
124,19590505,0.06,63,37,50.0,10.000000


In [4]:
data["YEAR"] = data["DATE"].astype(str).str.slice(0,4).astype(int)
data["MONTH"] = data["DATE"].astype(str).str.slice(4,6).astype(int)
data["YEARMONTH"] = data["DATE"].astype(str).str.slice(0,6).astype(int)

data.head()
data.dtypes


DATE           int64
PRCP         float64
TMAX           int32
TMIN           int32
TAVG         float64
TEMP_C       float64
YEAR           int32
MONTH          int32
YEARMONTH      int32
dtype: object

In [5]:
reference_temps = data.groupby(by="MONTH").mean()

In [6]:
reference_temps = reference_temps.reset_index()
reference_temps = reference_temps[["MONTH", "TAVG", "TEMP_C"]]
reference_temps["ref_temp"] = reference_temps["TEMP_C"]
reference_temps = reference_temps.drop(columns=["TEMP_C"])
reference_temps.head()

,MONTH,TAVG,ref_temp
0,1,5.566282,-14.685399
1,2,6.570752,-14.127360
2,3,14.811130,-9.549372
3,4,26.387202,-3.118221
4,5,38.990657,3.883698


In [7]:
temp_avg = data.groupby(by="YEARMONTH")[["TEMP_C", "MONTH"]].mean()
temp_avg = temp_avg.reset_index()
temp_avg.head()

,YEARMONTH,TEMP_C,MONTH
0,195905,5.259857,5.0
1,195906,11.157407,6.0
2,195907,12.750000,7.0
3,195908,11.424731,8.0
4,195909,3.796296,9.0


In [8]:
join = pd.merge(temp_avg, reference_temps, on="MONTH", how="left")
join.head()
join["DIFF"] = join["TEMP_C"] - join["ref_temp"]
join.head()

,YEARMONTH,TEMP_C,MONTH,TAVG,ref_temp,DIFF
0,195905,5.259857,5.0,38.990657,3.883698,1.376158
1,195906,11.157407,6.0,50.698851,10.388250,0.769157
2,195907,12.750000,7.0,56.378965,13.543869,-0.793869
3,195908,11.424731,8.0,51.756674,10.975930,0.448801
4,195909,3.796296,9.0,42.343860,5.746589,-1.950292


In [9]:
data_helsinki = pd.read_csv("data/merge_helsinki.csv")
data_helsinki.head()

,Unnamed: 0,YEARMONTH,TEMP_C_Helsinki,MONTH,TAVG_Helsinki,ref_temp_Helsinki,DIFF_Helsinki
0,0,195201,-0.416667,1.0,21.154787,-6.025118,5.608452
1,1,195202,-3.638889,2.0,20.899016,-6.167213,2.528324
2,2,195203,-10.895062,3.0,27.792892,-2.337282,-8.557779
3,3,195204,-0.277778,4.0,38.546655,3.637031,-3.914809
4,4,195205,5.555556,5.0,50.077766,10.043203,-4.487648


In [10]:
new_names = {"TEMP_C": "TEMP_C_Sodlokka", "TAVG": "TAVG_Sodlokka", "ref_temp": "ref_temp_Sodlokka", "DIFF": "DIFF_Sodlokka"}

In [11]:
data_sodlokka = join.rename(columns=new_names)

In [12]:
data_sodlokka.head()

,YEARMONTH,TEMP_C_Sodlokka,MONTH,TAVG_Sodlokka,ref_temp_Sodlokka,DIFF_Sodlokka
0,195905,5.259857,5.0,38.990657,3.883698,1.376158
1,195906,11.157407,6.0,50.698851,10.388250,0.769157
2,195907,12.750000,7.0,56.378965,13.543869,-0.793869
3,195908,11.424731,8.0,51.756674,10.975930,0.448801
4,195909,3.796296,9.0,42.343860,5.746589,-1.950292


In [22]:
compare_month = pd.merge(data_sodlokka, data_helsinki, on="YEARMONTH", how="outer")
compare_month = compare_month.dropna()
compare_month.head()
compare_month.dtypes
compare_month["MONTH"] = compare_month["MONTH_x"].astype(int)
compare_month.head()

,YEARMONTH,TEMP_C_Sodlokka,MONTH_x,TAVG_Sodlokka,ref_temp_Sodlokka,DIFF_Sodlokka,Unnamed: 0,TEMP_C_Helsinki,MONTH_y,TAVG_Helsinki,ref_temp_Helsinki,DIFF_Helsinki,MONTH
0,195905,5.259857,5.0,38.990657,3.883698,1.376158,88.0,9.390681,5.0,50.077766,10.043203,-0.652522,5
1,195906,11.157407,6.0,50.698851,10.388250,0.769157,89.0,14.777778,6.0,58.565072,14.758374,0.019404,6
2,195907,12.750000,7.0,56.378965,13.543869,-0.793869,90.0,17.338710,7.0,62.914712,17.174840,0.163870,7
3,195908,11.424731,8.0,51.756674,10.975930,0.448801,91.0,16.836918,8.0,60.345333,15.747407,1.089510,8
4,195909,3.796296,9.0,42.343860,5.746589,-1.950292,92.0,8.722222,9.0,51.323841,10.735467,-2.013245,9


In [47]:
month_summer = compare_month
 
month_summer = compare_month[(compare_month['MONTH'] >= 6) & (compare_month['MONTH'] <= 8)]
month_summer.head()
month_summer = month_summer.groupby(by="MONTH")[["ref_temp_Sodlokka", "ref_temp_Helsinki"]].mean().reset_index()
month_summer["DIFF"] = month_summer["ref_temp_Helsinki"] - month_summer["ref_temp_Sodlokka"]
month_summer.head()


,MONTH,ref_temp_Sodlokka,ref_temp_Helsinki,DIFF
0,6,10.388250,14.758374,4.370123
1,7,13.543869,17.174840,3.630971
2,8,10.975930,15.747407,4.771477


In [48]:
print(month_summer.mean())

MONTH                 7.000000
ref_temp_Sodlokka    11.636017
ref_temp_Helsinki    15.893540
DIFF                  4.257524
dtype: float64


In [49]:
print(month_summer.std())

MONTH                1.000000
ref_temp_Sodlokka    1.678174
ref_temp_Helsinki    1.214843
DIFF                 0.578531
dtype: float64


In [50]:
month_summer.to_csv("data/summer_month.csv")